In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

In [ ]:
import selenium
from selenium import webdriver as wd
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
from itertools import repeat
import re

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Referer': 'https://www.musinsa.com/',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8'
}

# 세션을 사용하면 쿠키 관리에 도움됨
session = requests.Session()

url = 'https://www.musinsa.com/brand/musinsastandard?gf=A'
response = session.get(url, headers=headers)

# 상태 코드 확인
print(response.status_code)

# 인코딩 명시적 설정
response.encoding = 'utf-8'

soup = BeautifulSoup(response.text, 'lxml')

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# WebDriver 생성
driver = webdriver.Chrome(options=chrome_options)

# 데이터 저장할 리스트 초기화
product_data = []

try:
    # 페이지 로드
    driver.get('https://www.musinsa.com/brand/musinsastandard?gf=A')

    # 명시적 대기 - 상품 목록이 로드될 때까지 대기
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.sc-fHejqy'))
    )

    # 충분한 로딩 시간 추가
    time.sleep(3)

    # 페이지 소스 가져오기
    page_source = driver.page_source

    # BeautifulSoup으로 파싱
    soup = BeautifulSoup(page_source, 'lxml')

    # 링크 찾기
    links = soup.find_all('a', class_=lambda x: x and 'sc-fHejqy' in x)

    # 각 링크에서 정보 추출
    for link in links:
        try:
            # 제품 URL
            product_url = link.get('href', '')

            # 제품 이미지
            img = link.find('img')
            # When you find the image tag
            if img:
                product_title = img.get('alt', '')

            # 가격 정보 추출 (데이터 속성 활용)
            original_price = link.get('data-original-price', '')
            discounted_price = link.get('data-price', '')
            discount_rate = link.get('data-discount-rate', '')

            # 브랜드 정보
            brand = link.get('data-item-brand', '')

            # 데이터 저장
            product_data.append({
                'title': product_title,
                'url': product_url,
                'original_price': original_price,
                'discounted_price': discounted_price,
                'discount_rate': discount_rate,
                'brand': brand
            })

        except Exception as e:
            print(f"Error processing link: {e}")

    # 데이터프레임으로 변환
    df = pd.DataFrame(product_data)

    # CSV로 저장
    df.to_csv('musinsastandard_products.csv', index=False, encoding='utf-8-sig')

    # 결과 출력
    print(f"Total products collected: {len(product_data)}")
    print("Data saved to musinsastandard_products.csv")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()

In [ ]:
df = pd.read_csv('/content/musinsastandard_products.csv')

In [ ]:
df